In [ ]:
# Import the required libraries
import torch
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import os
import tqdm

In [ ]:

# Set the CUDA_VISIBLE_DEVICES environment variable to specify GPU devices
# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"  # Use GPU devices 0 and 1

from flask import Flask, request, jsonify
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
import torch
import time
import re 


In [ ]:
data_final_test= pd.read_csv("data_final_test.csv")
len(data_final_test)


In [ ]:
data_final_test

In [ ]:
model_id="LLaMa2_13B_Chat-finetuned-New_annotations_v10"
# model_id="llama-2-13b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# Load the model and the tokenizer. Set generation config
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto")


generation_config = GenerationConfig(
    do_sample=True,
    top_k=1,
    temperature=0.1,
    max_new_tokens=300,
    pad_token_id=tokenizer.eos_token_id
)

In [ ]:
def solve_question(question_prompt):
    inputs = tokenizer(question_prompt, return_tensors="pt", padding=True, truncation=True,max_length= 2048).to("cuda")
    outputs = model.generate(**inputs, generation_config=generation_config)
    answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return answer

In [ ]:
all_answers = []
val_data_prompts = list(data_final_test['text'])

for i in tqdm.tqdm(range(0, len(val_data_prompts), 1)):
    question_prompts = val_data_prompts[i:i+1]
    ans = solve_question(question_prompts)

    all_answers.extend(ans)
    torch.cuda.empty_cache()

In [ ]:
all_answers

In [ ]:
len(all_answers)

In [ ]:
data_final_test['Result_Finetuned'] = all_answers
data_final_test.to_csv('data_final_test.csv', index=False)
data_final_test